In [1]:
from py2neo import Graph
from pandas import DataFrame
from pprint import pprint

# Test

In [2]:
graph = Graph(host="localhost://7474", auth=("neo4j", "123456"))

In [13]:
query = """
        MATCH (r:Review)-[a:About]->(re:Restaurant)
        WHERE re.cuisine = 'western'
        WITH re,COUNT(r) AS number,SUM(a.restaurant_sentiment) AS senti      
        
        RETURN re.name, senti/number AS average_senti, number
        ORDER BY average_senti DESC

"""

a=graph.run(query).data()
a

[{'re.name': 'Jones the Grocer (TripleOne Somerset)',
  'average_senti': 0.907778289464286,
  'number': 28},
 {'re.name': '49 Seats (The Centrepoint)',
  'average_senti': 0.9058165971600001,
  'number': 25},
 {'re.name': 'Little Creatures',
  'average_senti': 0.9024693337999999,
  'number': 30},
 {'re.name': "Meat n' Chill",
  'average_senti': 0.895583758488889,
  'number': 90},
 {'re.name': 'Original Sin',
  'average_senti': 0.8939380756666665,
  'number': 30},
 {'re.name': 'Coastes', 'average_senti': 0.8931061993787877, 'number': 66},
 {'re.name': 'Drunken Farmer',
  'average_senti': 0.8902770238235295,
  'number': 17},
 {'re.name': 'Cheeky', 'average_senti': 0.8864718550625, 'number': 48},
 {'re.name': 'Miki Kitchen - Oriental Cuisine',
  'average_senti': 0.8859556606666668,
  'number': 18},
 {'re.name': 'Wholesome Savour',
  'average_senti': 0.8857972805294119,
  'number': 17},
 {'re.name': "Wheeler's Estate",
  'average_senti': 0.8845910152333333,
  'number': 90},
 {'re.name': 'AT

In [27]:
query_b = """
        MATCH (r:Review)-[c:Contains]->(f:Food)
        WHERE f.name = 'hotpot'
        WITH DISTINCT r, c
        MATCH (r)-[a:About]->(re:Restaurant)
        RETURN re.name, sum(c.food_sentiment)/count(r) AS average_senti, count(r) AS number
        ORDER BY number DESC
        """
b=graph.run(query_b).to_data_frame()
b

,average_senti,number,re.name
0,0.905860,12,Fat Bird (Bugis)
1,0.880257,11,Jiu Gong Ge Hotpot
2,0.865867,9,Guo Fu Steamboat
3,0.916393,8,Tamoya Udon (Liang Court)
4,0.858514,7,JiangHu Hotpot
5,0.818162,7,City Hot Pot Shabu Shabu (One Raffles Place)
6,0.855129,7,Four Seasons Durians Restaurant
7,0.910653,6,Shaburi & Kintan Buffet
8,0.858474,5,Tamashii Robataya
9,0.928669,4,Congee Legend Hotpot


In [89]:
#Query top 3 and Bottom 3 reviews for food
query_top = """
            MATCH (r:Review)-[a:About]->(re:Restaurant)
            WHERE re.cuisine = 'local' AND re.name = 'Moc Quan (UE Square)'
            WITH re, a, r
            ORDER BY a.restaurant_sentiment DESC
            WITH re, COLLECT(r)[0..3] AS top3
            UNWIND top3 AS top
            RETURN re.name, top.review
            """

query_bottom = """
            MATCH (r:Review)-[a:About]->(re:Restaurant)
            WHERE re.name = 'Fat Bird (Bugis)'
            WITH re, a, r
            MATCH (r:Review)-[c:Contains]->(f:Food)
            WHERE f.name = 'hotpot'
            WITH re, r, c
            ORDER BY c.food_sentiment DESC
            WITH re, COLLECT(r)[-4..-1] AS bottom3
            UNWIND bottom3 AS bottom
            RETURN re.name, bottom.review
            """
c=graph.run(query_top).to_data_frame()
c

,re.name,top.review
0,Moc Quan (UE Square),Bun Thit Nuong ( BBQ minced meat rice noodles ...
1,Moc Quan (UE Square),Not much of a looker but definitely makes for ...
2,Moc Quan (UE Square),"Bun Bo Hue / 14.9bucks:Rice noodles, slices of..."


In [13]:
def query_reviews_cuisine(res_list):
    result = []
    query_top = """
                MATCH (r:Review)-[a:About]->(re:Restaurant)
                WHERE re.name = {term}
                WITH re, a, r
                ORDER BY a.restaurant_sentiment DESC
                WITH re, COLLECT(r)[0..3] AS top3
                UNWIND top3 AS top
                RETURN re.name, top.review
                """

    query_bottom = """
                MATCH (r:Review)-[a:About]->(re:Restaurant)
                WHERE re.name = {term}
                WITH re, a, r
                ORDER BY a.restaurant_sentiment DESC
                WITH re, COLLECT(r)[-4..-1] AS top3
                UNWIND top3 AS top
                RETURN re.name, top.review
                """
    for item in res_list:
        
        temp = [item]
        tmp_t = graph.run(query_top, term=item).data()
        for r in tmp_t:
            temp.append(r['top.review'])
            
        tmp_b = graph.run(query_bottom, term=item).data()
        for r in tmp_b:
            temp.append(r['top.review'])
        
        result.append(temp)
        
    return result


query_reviews_cuisine(['Moc Quan (UE Square)','Fat Bird (Bugis)'])

[['Moc Quan (UE Square)',
  'Bun Thit Nuong ( BBQ minced meat rice noodles ) | Pho Bo | Banh Mi Roast Beef Meats are all grilled with charcoal and a special sauce that was so good.:Great find. 😋😋😋 . . . . . #vietnamesefood #banhmi #bunthitnuong #pho #burpple #sgfoodies #sgfood',
  'Not much of a looker but definitely makes for a tasty dinner.:Grilled pork with rice!',
  'Bun Bo Hue / 14.9bucks:Rice noodles, slices of beef, pork and beansprouts sitting in that rich broth made of beef and pork bones with a slight hint of lemongrass and Vietnamese coriander. Vietnamese comfort food at its best, definitely a favourite (and a rare one too) as those gems within Joo Chiat packed with authentic tasty Vietnamese fare. 14.9bucks/bowl #FTrecommends #mocquan',
  'Bun Thit Nuong:Bun Thit Nuong ($11.90) was pretty impressed by this dish. The rice vermicelli was light and not too “sticky”, generous amounts of greens were given (as with all authentic Vietnamese bun dishes), and the grilled pork was ex

In [16]:
def query_reviews_food(graph, res_list, food):
    result = []
    query_top = """
            MATCH (r:Review)-[a:About]->(re:Restaurant)
            WHERE re.name = {restaurant}
            WITH re, r
            MATCH (r:Review)-[c:Contains]->(f:Food)
            WHERE f.name = {food}
            WITH re, r, c
            ORDER BY c.food_sentiment DESC
            WITH re, COLLECT(r)[0..3] AS top3
            UNWIND top3 AS top
            RETURN re.name, top.review
            """

    query_bottom = """
            MATCH (r:Review)-[a:About]->(re:Restaurant)
            WHERE re.name = {restaurant}
            WITH re, r
            MATCH (r:Review)-[c:Contains]->(f:Food)
            WHERE f.name = {food}
            WITH re, r, c
            ORDER BY c.food_sentiment DESC
            WITH re, COLLECT(r)[-4..-1] AS top3
            UNWIND top3 AS top
            RETURN re.name, top.review
            """
    for item in res_list:
        
        temp = [item]
        tmp_t = graph.run(query_top, restaurant=item, food=food).data()
        for r in tmp_t:
            temp.append(r['top.review'])
            
        tmp_b = graph.run(query_bottom, restaurant=item, food=food).data()
        for r in tmp_b:
            temp.append(r['top.review'])
        
        result.append(temp)
        
    return result

query_reviews_food(graph,['Jiu Gong Ge Hotpot','Fat Bird (Bugis)'],'hotpot')

[['Jiu Gong Ge Hotpot',
  'Hotpot + BBQ:A little bit of something for everybody.. for hotpot lovers, BBQ lovers, meat lovers & seafood lovers ...taste.. not bad! Pretty OK!! Went during lunch time additional charges for the drinks',
  "Seafood Platter For 4:Happy birthday Daddy <3 This humongous platter ($118) covered the entire table and was filled with a clear soup which was rich with the sweetness from the seafood. Swimming in this huge hotpot were three crabs, half a lobster, prawns, some crayfish-prawn crustacean, mussels, clams, oysters and scallops. There were also quail eggs, corn and cabbage as well as instant noodles. My favorite were hands-down the prawns which were juicy and crunchy. The shell peeled easily away from the tender bright orange flesh, a sure sign of its freshness. The lobster was the biggest disappointment however, chewy and overcooked. :( The soup was an absolute delight to drink, brimming with seafoody goodness. I was thankful for that fact that we hadn't go

# Search query functions

In [98]:
def search_restaurant_by_food(graph,text,sort):
    result=[]
    key_words = text
    result_review={}
    
    query_rating = """
        MATCH (r:Review)-[c:Contains]->(f:Food)
        WHERE f.name = {term}
        WITH DISTINCT r, c
        MATCH (r)-[a:About]->(re:Restaurant)
        RETURN re.name, sum(c.food_sentiment)/count(r) AS average_senti, count(r) AS number
        ORDER BY average_senti DESC
        """
    query_popularity = """
        MATCH (r:Review)-[c:Contains]->(f:Food)
        WHERE f.name = {term}
        WITH DISTINCT r, c
        MATCH (r)-[a:About]->(re:Restaurant)
        RETURN re.name, sum(c.food_sentiment)/count(r) AS average_senti, count(r) AS number
        ORDER BY number DESC
        """
    
    query_top = """
            MATCH (r:Review)-[a:About]->(re:Restaurant)
            WHERE re.name = {restaurant}
            WITH re, a, r
            MATCH (r:Review)-[c:Contains]->(f:Food)
            WHERE f.name = {food}
            WITH re, r, c
            ORDER BY c.food_sentiment DESC
            WITH re, COLLECT(r)[0..3] AS top3
            UNWIND top3 AS top
            RETURN re.name, top.review
            """

    query_bottom = """
                MATCH (r:Review)-[a:About]->(re:Restaurant)
                WHERE re.name = {restaurant}
                WITH re, a, r
                MATCH (r:Review)-[c:Contains]->(f:Food)
                WHERE f.name = {food}
                WITH re, r, c
                ORDER BY c.food_sentiment DESC
                WITH re, COLLECT(r)[-4..-1] AS bottom3
                UNWIND bottom3 AS bottom
                RETURN re.name, bottom.review
                """

    if sort == 'number_of_reviews':
        query = query_popularity

    if sort == 'rating':
        query = query_rating
        
    ans = graph.run(query,term=text).data()

    if ans:
        for item in ans:
            result.append(item['re.name'])
            result_review[item['re.name']]=[]
            
        for res in result:
            t = []
            b = []

            temp_t = graph.run(query_top, restaurant=res, food=key_words).data()
            for element in temp_t:
                result_review[element['re.name']].append(element['top.review'])
                
            temp_b = graph.run(query_bottom, restaurant=res, food=key_words).data()
            for element in temp_b:
                result_review[element['re.name']].append(element['bottom.review'])
                
        return result_review
    
    if not ans:
        key_words = key_words.split()
        result={}
        
        for i in range(len(key_words)):
            temp = graph.run(query, term=key_words[i]).data()
            result[key_words[i]]=[]
            for item in temp:
                result[key_words[i]].append(item['re.name'])
                
        if not result:
            return ['No records found']
        else:
            for res in result:
                t = []
                b = []

                temp_t = graph.run(query_top, restaurant=res, food=key_words).data()
                for element in temp_t:
                    result_review[element['re.name']].append(element['top.review'])

                temp_b = graph.run(query_bottom, restaurant=res, food=key_words).data()
                for element in temp_b:
                    result_review[element['re.name']].append(element['bottom.review'])
                
            return result_review
        

def search_restaurant_by_cusine(graph,text,sort):
    result=[]
    result_review={}
    
    query_rating = """
        MATCH (r:Review)-[a:About]->(re:Restaurant)
        WHERE re.cuisine = {term}
        WITH re,COUNT(r) AS number,SUM(a.restaurant_sentiment) AS senti      
        
        RETURN re.name, senti/number AS average_senti
        ORDER BY average_senti DESC

    """
    
    query_popularity = """
        MATCH (r:Review)-[a:About]->(re:Restaurant)
        WHERE re.cuisine = {term}
        WITH re,COUNT(r) AS number        
        RETURN re.name,number
        ORDER BY number DESC

    """
    
    query_top = """
            MATCH (r:Review)-[a:About]->(re:Restaurant)
            WHERE re.name ={term}
            WITH re, a, r
            ORDER BY a.restaurant_sentiment DESC
            WITH re, COLLECT(r)[0..3] AS top3
            UNWIND top3 AS top
            RETURN re.name, top.review
            """
    
    query_bottom = """
            MATCH (r:Review)-[a:About]->(re:Restaurant)
            WHERE re.name = {term}
            WITH re, a, r
            ORDER BY a.restaurant_sentiment DESC
            WITH re, COLLECT(r)[-4..-1] AS bottom3
            UNWIND bottom3 AS bottom
            RETURN re.name, bottom.review
            """
    if sort == 'number_of_reviews':
        query = query_popularity
    
    if sort == 'rating':
        query = query_rating
        
    ans = graph.run(query, term=text).data()
    if not ans:
        return ['No records found']
    else: 
        for item in ans:
            result.append(item['re.name'])
            result_review[item['re.name']]=[]
            
        for res in result:
            t = []
            b = []

            temp_t = graph.run(query_top, term=res).data()
            for element in temp_t:
                result_review[element['re.name']].append(element['top.review'])
                
            temp_b = graph.run(query_bottom, term=res).data()
            for element in temp_b:
                result_review[element['re.name']].append(element['bottom.review'])
                
        return result_review

    



In [99]:
a=search_restaurant_by_cusine(graph, 'local','number_of_reviews' )

In [106]:
for item in a:
    print('/n')
    print(item)

/n
Chinatown Complex Market & Food Centre
/n
Swee Choon Dim Sum
/n
Tiong Bahru Market
/n
85 Fengshan Centre (Bedok 85)
/n
Maxwell Food Centre
/n
GRUB (Bishan)
/n
Dumpling Darlings
/n
KEK Keng Eng Kee Seafood (Alexandra)
/n
Victor's Kitchen (Sunshine Plaza)
/n
Chomp Chomp Food Centre
/n
The Salted Plum (Circular Road)
/n
Redhill Lane Block 85 Food Centre
/n
Ghim Moh Market & Food Centre
/n
Hill Street Tai Hwa Pork Noodle
/n
Kok Sen Restaurant
/n
Malaysia Boleh! (Jurong Point)
/n
Tong Ah Eating House
/n
Penang Seafood Restaurant
/n
Lau Pa Sat (Telok Ayer Market)
/n
The Coconut Club
/n
Legendary Hong Kong
/n
Albert Centre Market & Food Centre
/n
Beach Road Scissors Cut Curry Rice (Jalan Besar)
/n
Dim Sum Haus
/n
Som Tam
/n
Food Republic (Wisma Atria)
/n
Enjoy Eating House and Bar
/n
Kko Kko Na Ra (Tanjong Pagar)
/n
Kim Keat Hokkien Mee
/n
WANTON Seng's Noodle Bar
/n
Mui Kee Congee
/n
Mouth Restaurant (Maxwell)
/n
YY Kafei Dian
/n
Springleaf Prata Place (Jalan Tua Kong)
/n
Ah Loy Thai
/n
N

In [67]:
a=[{'re.name': 'Fat Bird (Bugis)', 'top.review': 'Fragrant Chicken Hotpot $20++:Used Burpple Beyond, ordered small hotpot 1-for-1. House special chicken is somewhat like gong bao chicken, delicious gravy not too overpowering, perfect with steam rice. Spicy & fragrant chicken is a mild Mala version. If not big eaters, 2 small hotpots can feed 3 pax (or even 4). Perfect for our family of 2+2 children'}, {'re.name': 'Fat Bird (Bugis)', 'top.review': "Potful of Goodness:Had the signature hotpot chicken, with enoki mushroom and quail eggs for lunch, and boy, was it good! My lunch kakis finished the pot, and ordered more rice to slurp up every last bit of the delish gravy. Oh, and we had the crispy duck (👍), string beans with minced meat (👍) and some bacon dish (👍). All in all, extremely satisfying, made particularly better with good company. Can't wait to visit again and try the other offerings."}, {'re.name': 'Fat Bird (Bugis)', 'top.review': 'Fat Bird Location 🗺: Heartbeat @ Bedok (Permanently Closed) MRT 🚇: Bedok Opening Hours 🕒: 10:30AM – 12AM .:Rating 📈: 4.5/5 Price 💸: $15.90++ Review 💬: One of my favourite mala places because their mala was exactly how i like it, spicy but not too numb - where you cannot feel your tongue. Although it ended up costing almost $19 per pax, the fact that it is a buffet makes it very worth it. Some ingredients include you mai 菜，pork slices, meat balls, enoki mushrooms, maggi mee and even fried mantous. The mala hotpot base we got was the one which included the bullfrog, although yummy, the meat to bone ratio was 1:3. Nevertheless, food was great and so was service from the guy with the little ponytail. Sad to hear that they’ve closed down this particular outlet, but will definitely visit their other outlets. #euniceeatssavoury #sgigfoodies #burpple'}]

In [70]:
a[0]

{'re.name': 'Fat Bird (Bugis)',
 'top.review': 'Fragrant Chicken Hotpot $20++:Used Burpple Beyond, ordered small hotpot 1-for-1. House special chicken is somewhat like gong bao chicken, delicious gravy not too overpowering, perfect with steam rice. Spicy & fragrant chicken is a mild Mala version. If not big eaters, 2 small hotpots can feed 3 pax (or even 4). Perfect for our family of 2+2 children'}